In [11]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [19]:
lstm = nn.LSTM(3,3)
inputs = [autograd.Variable(torch.randn(1, 3))
          for _ in range(5)]  # make a sequence of length 5
hidden = (
          autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn(1, 1, 3))
         )
for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)


(Variable containing:
(0 ,.,.) = 
  0.3144  0.4067 -0.2899
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  1.0458  1.0092 -0.6132
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
  0.2405  0.1165 -0.2440
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  0.2758  1.0055 -0.4459
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
  0.1722  0.1892 -0.1067
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  0.2547  0.9995 -0.2025
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
  0.1509  0.2296 -0.1337
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  0.2697  1.0722 -0.2731
[torch.FloatTensor of size 1x1x3]
)
(Variable containing:
(0 ,.,.) = 
  0.2806  0.4155  0.0818
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  0.5521  0.9221  0.1809
[torch.FloatTensor of size 1x1x3]
)


In [20]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

In [21]:
class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(BATCH_SIZE, 1, hidden_dim)),
                autograd.Variable(torch.zeros(BATCH_SIZE, 1, hidden_dim)))

    def forward(self, sequence):
        lstm_out, self.hidden = self.lstm(word, self.hidden)
        return self.hidden


In [22]:
EMBEDDING_DIM, HIDDEN_DIM = 200, 200
lstm = LSTM(EMBEDDING_DIM, HIDDEN_DIM)

for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(train_loader):
        #Sample shape: [50, 22, 200, 38]
        #50 - Batch size, 22 - Num questions per data point
        batch_cos_similarities = []
        batch_num = idx
        
        #CNN : BATCH X WORD EMBEDDINGS X WORD
        #LSTM : WORD X BATCH X WORD EMBEDDING
        
        target_question_features = sample[:, 0]
        positive_question_features = sample[:, 1]
        N_negative_question_features = sample[:, 2:]
        
        target_question_lstm_output = lstm(Variable(target_question_features))
        positive_question_lstm_output = lstm(Variable(positive_question_features))
        N_negative_question_lstm_outputs = [lstm(Variable(negative)) for negative in N_negative_question_features]
        
        #Determine lengths to know how many vectors to take the average across.
        target_question_lengths = find_start_of_padding_for_batch(target_question_features)
        positive_question_lengths = find_start_of_padding_for_batch(positive_question_features)
        N_negative_questions_lengths = [find_start_of_padding_for_batch(negative) for negative in N_negative_question_features]
                
        batch_cos_sims = []
        #50x600x38 --> 50x600x1. Averaging across the lstm outputs to get a single vector for each question.
        for sample_num in range(BATCH_SIZE):
            target_question_length = target_question_lengths[sample_num]
            positive_question_length = positive_question_lengths[sample_num]
            N_negative_question_lengths = N_negative_questions_lengths[sample_num]
            N_negative_question_outputs = N_negative_question_lstm_outputs[sample_num]
            
            target_question_vector = target_question_lstm_output.data.numpy()[sample_num,:,:target_question_length].mean(axis = 1)
            positive_question_vector = positive_question_lstm_output.data.numpy()[sample_num,:,:positive_question_length].mean(axis = 1)
            
            N_negative_question_vectors = [negative.data.numpy()[:,:N_negative_question_lengths[idx]].mean(axis = 1) for idx, negative in enumerate(N_negative_question_outputs)]
        
            cosine_similarity_pos = np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), positive_question_vector.reshape(1, -1)))
            cosine_similarity_neg = [np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), negative.reshape(1, -1))) for negative in N_negative_question_vectors]
            
            cosine_sims = np.array([cosine_similarity_pos] + cosine_similarity_neg).flatten()
            batch_cos_sims.append(cosine_sims)
        
        optimizer.zero_grad()
        
        batch_labels = Variable(torch.from_numpy(label.numpy().squeeze()), requires_grad = True)
        loss = criterion(Variable(torch.FloatTensor(np.array(batch_cos_sims))), batch_labels)
            
        loss.backward()
            
        running_loss += loss.data[0]
            
        optimizer.step()
        print "Epoch: {}, Batch: {}, Loss: {}".format(epoch, batch_num, loss.data[0])

    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
    


NameError: global name 'BATCH_SIZE' is not defined